In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [6]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5), padding=2)
        self.avg_pool_1 = nn.AvgPool2d(kernel_size=(2, 2), stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5))
        self.avg_pool_2 = nn.AvgPool2d(kernel_size=(2, 2), stride=2)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.sigmoid(self.conv1(x))
        x = self.avg_pool_1(x)
        x = F.sigmoid(self.conv2(x))
        x = self.avg_pool_2(x)
        x = x.view(-1, 400)
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = LeNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [7]:
transform = transforms.ToTensor()

# figure out how to import the data that has already been downloaded in the MLP folder, but include other code for downloading regularly.

"""
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_dataset, val_dataset = random_split(train_dataset, [55000, 5000])
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
"""

train_dataset = datasets.MNIST(root='../mnistMLP/data', train=True, download=True, transform=transform)
train_dataset, val_dataset = random_split(train_dataset, [55000, 5000])
test_dataset = datasets.MNIST(root='../mnistMLP/data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
def validate(model, val_loader):
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    val_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print(f'Validation set: Average loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

def train(epoch, log_interval=200):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
        
num_epochs = 5

for epoch in range(1, num_epochs + 1):
    train(epoch)
    validate(model, val_loader)

validate(model, test_loader)

Train Epoch: 1 [0/55000 (0%)]	Loss: 2.286179
Train Epoch: 1 [6400/55000 (12%)]	Loss: 2.311466
Train Epoch: 1 [12800/55000 (23%)]	Loss: 1.099192
Train Epoch: 1 [19200/55000 (35%)]	Loss: 0.871420
Train Epoch: 1 [25600/55000 (47%)]	Loss: 0.412893
Train Epoch: 1 [32000/55000 (58%)]	Loss: 0.489651
Train Epoch: 1 [38400/55000 (70%)]	Loss: 0.209047
Train Epoch: 1 [44800/55000 (81%)]	Loss: 0.145969
Train Epoch: 1 [51200/55000 (93%)]	Loss: 0.512564
Validation set: Average loss: 0.0082, Accuracy: 92.52%
Train Epoch: 2 [0/55000 (0%)]	Loss: 0.563912
Train Epoch: 2 [6400/55000 (12%)]	Loss: 0.135127
Train Epoch: 2 [12800/55000 (23%)]	Loss: 0.087854
Train Epoch: 2 [19200/55000 (35%)]	Loss: 0.108780
Train Epoch: 2 [25600/55000 (47%)]	Loss: 0.210068
Train Epoch: 2 [32000/55000 (58%)]	Loss: 0.230399
Train Epoch: 2 [38400/55000 (70%)]	Loss: 0.183256
Train Epoch: 2 [44800/55000 (81%)]	Loss: 0.438899
Train Epoch: 2 [51200/55000 (93%)]	Loss: 0.105802
Validation set: Average loss: 0.0051, Accuracy: 95.50%
Tr